In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/BUSD LP strategy
---


In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [3]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure, plot_price_ratio

mkt_pair = 'LUNC/BUSD'
start, end = '2022-10-01 00:00:00', '2023-04-25 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

qty_factor = 0.001
df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = qty_factor*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=800)))

In [4]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote("LUNC/USD", df_trades.price.iloc[0])
# LUNC/BUSD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 100000 USD market for LUNC/USTC
mkt = new_market(liquidity_usd, quote, base, 0.003)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# uni v2 lp strategy with arb to keep the price of the pool in line with mkt
strategy = SimpleUniV2Strategy(cp_amm, arb_enabled=True)
# run simulation
simul = swap_simulation(df_trades, strategy)
# display sim results
show(new_simulation_figure(mkt, simul))

Function 'trade_summary' executed in 0.0208s
Function 'sim_results' executed in 0.2772s
Function 'swap_simulation' executed in 1.0719s


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

simul["breakdown"].head(100)

,side,price,price_impact,qty_received,mid_price,mkt_price,spread,avg_price,current_base,current_quote,cp_invariant,total_fees_paid_quote,total_volume_base,total_volume_quote,asset_base_pct,hold_portfolio,current_portfolio,trade_pnl,total_pnl,roi,impermanent_loss,mkt_price_ratio,arb_profit,volume_base,volume_quote,fees_paid_quote,trade_pnl_pct,fees_pnl_pct,total_arb_profit
trade_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-01 00:00:00,sell,0.00030,0.00268,"1,339.86171",0.00030,0.00030,0.00000,0.00030,"1,657,881,818.18767","498,660.13829","826,719,576,719,576.87500",4.03168,"4,442,664.74851","-1,343.89339",0.50000,"997,327.89871","997,320.27657",-7.62214,-3.59046,-0.00000,0.00000,1.00000,0.00000,"4,442,664.74851","-1,343.89339",4.03168,-0.00001,0.00000,0.00000
2022-10-01 01:00:00,sell,0.00030,0.00539,"2,686.25442",0.00030,0.00030,-0.00000,0.00030,"1,666,861,106.23522","495,973.88386","826,719,576,719,576.75000",12.11469,"13,421,952.79606","-4,038.23083",0.50000,"991,992.30164","991,947.76773",-44.53391,-32.41922,-0.00003,-0.00003,0.98393,"48,925.14308","8,979,288.04755","-2,694.33744",8.08301,-0.00004,0.00001,"48,925.14308"
2022-10-01 01:00:00,sell,0.00030,0.00171,847.09588,0.00030,0.00030,0.00000,0.00030,"1,669,712,883.20474","495,126.78799","826,719,576,719,576.75000",14.66363,"16,273,729.76559","-4,887.87564",0.50000,"990,315.73599","990,253.57597",-62.16002,-47.49639,-0.00005,-0.00003,0.98393,0.00000,"2,851,776.96952",-849.64481,2.54893,-0.00006,0.00001,"48,925.14308"
2022-10-01 02:00:00,buy,0.00030,-0.00482,"8,014,436.35039",0.00030,0.00030,0.00000,0.00030,"1,661,698,446.85435","497,514.80377","826,719,576,719,576.87500",21.84923,"8,259,293.41520","-2,492.67425",0.50000,"995,049.43797","995,029.60755",-19.83042,2.01881,0.00000,-0.00001,0.99011,11.58670,"-8,014,436.35039","2,395.20139",7.18560,-0.00002,0.00002,"48,936.72979"
2022-10-01 02:00:00,buy,0.00030,-0.00044,"734,253.98436",0.00030,0.00030,-0.00000,0.00030,"1,660,964,192.86999","497,734.73761","826,719,576,719,576.75000",22.51102,"7,525,039.43083","-2,272.07862",0.50000,"995,486.55429","995,469.47522",-17.07906,5.43195,0.00001,-0.00001,0.99011,0.00000,"-734,253.98436",220.59562,0.66179,-0.00002,0.00002,"48,936.72979"
2022-10-01 03:00:00,sell,0.00030,0.00226,"1,124.16687",0.00030,0.00030,-0.00000,0.00030,"1,664,724,082.46882","496,610.57074","826,719,576,719,576.75000",25.89367,"11,284,929.02966","-3,399.62814",0.50000,"993,254.31683","993,221.14148",-33.17535,-7.28168,-0.00001,-0.00002,0.98647,"8,562.32645","3,759,889.59883","-1,127.54952",3.38265,-0.00003,0.00003,"57,499.05624"
2022-10-01 03:00:00,buy,0.00030,-0.00014,"238,049.71740",0.00030,0.00030,-0.00000,0.00030,"1,664,486,032.75142","496,681.59447","826,719,576,719,576.75000",26.10738,"11,046,879.31226","-3,328.39070",0.50000,"993,395.19774","993,363.18894",-32.00880,-5.90142,-0.00001,-0.00002,0.98647,0.00000,"-238,049.71740",71.23744,0.21371,-0.00003,0.00003,"57,499.05624"
2022-10-01 04:00:00,sell,0.00030,0.00783,"3,889.80074",0.00029,0.00029,-0.00000,0.00030,"1,677,624,480.03374","492,791.79373","826,719,576,719,576.87500",37.81189,"24,185,326.59459","-7,229.89596",0.50000,"985,709.19362","985,583.58746",-125.60616,-87.79427,-0.00009,-0.00011,0.97133,"104,330.14151","13,138,447.28232","-3,901.50526",11.70452,-0.00013,0.00004,"161,829.19775"
2022-10-01 04:00:00,sell,0.00029,0.00048,236.74163,0.00029,0.00029,0.00000,0.00030,"1,678,430,813.36676","492,555.05210","826,719,576,719,576.87500",38.52426,"24,991,659.92761","-7,467.34995",0.50000,"985,243.36074","985,110.10419",-133.25655,-94.73229,-0.00010,-0.00011,0.97133,0.00000,"806,333.33302",-237.45399,0.71236,-0.00014,0.00004,"161,829.19775"
